# Profile

Nama : Chandra Arifin <br/>
Project : Submission 1 <br/>
Machine Learning

# Import Libraries

In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Input

# Baca File dan Hapus Kolom

In [2]:
df = pd.read_csv('movies.csv')

In [3]:
#penghapusan kolom yang tidak dipakai
df = df.drop(columns=['year', 'released', 'budget', 'gross', 'director', 'writer', 'star', 'country', 'company', 'runtime', 'votes',
                      'rating', 'score'])

In [4]:
#tampilkan record yang ada
df.head()

,name,genre
0,The Shining,Drama
1,The Blue Lagoon,Adventure
2,Star Wars: Episode V - The Empire Strikes Back,Action
3,Airplane!,Comedy
4,Caddyshack,Comedy


# Pemisahan Kategori dan Penyatuan Data

In [5]:
#ambil kategori yang ada di film
category = pd.get_dummies(df.genre)

In [6]:
newDF = pd.concat([df, category], axis=1)

In [7]:
newDF = newDF.drop(columns='genre')
newDF

,name,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,Western
0,The Shining,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,The Blue Lagoon,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Star Wars: Episode V - The Empire Strikes Back,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Airplane!,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Caddyshack,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7663,More to Life,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
7664,Dream Round,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7665,Saving Mbango,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
7666,It's Just Us,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
namaFilm = newDF['name'].values
val = newDF[['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
             'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Musical',
             'Mystery', 'Romance', 'Sci-Fi', 'Sport',	'Thriller', 'Western']].values

In [9]:
namaFilm

array(['The Shining', 'The Blue Lagoon',
       'Star Wars: Episode V - The Empire Strikes Back', ...,
       'Saving Mbango', "It's Just Us", 'Tee em el'], dtype=object)

In [10]:
val

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

# Pelatihan dan Test

In [11]:
namaFilm_latih, namaFilm_test, val_latih, val_test = train_test_split(namaFilm, val, test_size=0.2)

# Tokenizer

In [12]:
token = Tokenizer(num_words=3000, oov_token='x')
token.fit_on_texts(namaFilm_latih)
token.fit_on_texts(namaFilm_test)

In [13]:
seq_latih = token.texts_to_sequences(namaFilm_latih)
seq_test = token.texts_to_sequences(namaFilm_test)

pad_latih = pad_sequences(seq_latih)
pad_test = pad_sequences(seq_test)

# Pembuatan Model

In [14]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
#     tf.keras.layers.LSTM(64),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dense(5, activation='softmax')
# ])
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(3000, 100, input_length=namaFilm.shape[0]),
    # tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(7668, activation='softmax'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


# model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.Embedding(3000, 100, input_length=x.shape[1]))
# model.add(tf.keras.layers.LSTM(64))
# model.add(tf.keras.layers.Dense(32, activation='relu'))
# model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [15]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [16]:
model.fit(pad_latih, val_latih, epochs=30, validation_data=(pad_test, val_test), verbose=2)

Epoch 1/30


ValueError: ignored

In [ ]:
teks = ['Saya suka programming',
        'Programming sangat menyenangkan!',
        'Machine Learning berbeda dengan pemrograman konvensional']

In [ ]:
tokenizer.fit_on_texts(teks)

In [ ]:
sequences = tokenizer.texts_to_sequences(teks)

In [ ]:
print(tokenizer.word_index)

In [ ]:
sequences_samapanjang = pad_sequences(sequences)

In [ ]:
sequences_samapanjang = pad_sequences(sequences,
                                      padding='post',
                                      maxlen=5)

In [ ]:

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(3000, 100, input_length=x.shape[1]))
model.add(tf.keras.layers.LSTM(64))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:


model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(jumlah_kata, dimensi_embedding, panjang_input),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.95):
      print("\nAkurasi telah mencapai >95%!")
      self.model.stop_training = True
callbacks = myCallback()

In [ ]:
hist = model.fit(x, y, epochs=20, validation_data=(x_test, y_test), callbacks=[callbacks])

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(padded_latih, label_latih,
          epochs=num_epochs,
          validation_data=(padded_test, label_test))

# New Section

In [ ]:
import pandas as pd
df = pd.read_csv('imdb_indonesian_movies_2.csv')
df = df.drop(columns=['judul_film'])

In [ ]:
df.head()

In [ ]:
category = pd.get_dummies(df.genre)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='genre')
df_baru

In [ ]:
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

In [ ]:
sinopsis

In [ ]:
label

In [ ]:
from sklearn.model_selection import train_test_split
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(sinopsis_latih)
tokenizer.fit_on_texts(sinopsis_test)

sekuens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sekuens_test = tokenizer.texts_to_sequences(sinopsis_test)

padded_latih = pad_sequences(sekuens_latih)
padded_test = pad_sequences(sekuens_test)

In [ ]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs,
                    validation_data=(padded_test, label_test), verbose=2)